# Spark 101

In [138]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()

import pandas as pd
import numpy as np



from pyspark.sql.functions import *

# Note: The pyspark avg and mean functions are aliases of eachother
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import lit





## Exercise 1:
Within your codeup-data-science directory, create a new repo named spark-exercises. This will be where you do your work for this module. Create a repository on GitHub with the same name, and link your local repository to GitHub.

Save this work in your spark-exercises repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named spark101 for this exercise.

Create a spark data frame that contains your favorite programming languages.

Create a dataframe with one column named language

Hint: Start with a pandas dataframe. Maybe use a dictionary?
View the schema of the dataframe
Output the shape of the dataframe
Show the first 5 records in the dataframe

In [14]:
np.random.seed(123)
pandas_dataframe = pd.DataFrame(
    dict(language=['english','chicken','python','java','java_script']))
pandas_dataframe

,language
0,english
1,chicken
2,python
3,java
4,java_script


In [15]:
df = spark.createDataFrame(pandas_dataframe)
df.show(5)

+-----------+
|   language|
+-----------+
|    english|
|    chicken|
|     python|
|       java|
|java_script|
+-----------+



In [9]:
df.describe().show()

+-------+--------+
|summary|language|
+-------+--------+
|  count|      20|
|   mean|    null|
| stddev|    null|
|    min|       h|
|    max|       y|
+-------+--------+



In [12]:
print("DataFrame shape: ", df.count(), " x ", len(df.columns))

DataFrame shape:  20  x  1


In [17]:
df.printSchema()

root
 |-- language: string (nullable = true)



# Exercise 2:
Load the mpg dataset as a spark dataframe.
a. Create 1 column of output that contains a message like the one below for each record:
The 1999 audi a4 has a 4 cylinder engine.
Hint: You will need to concatenate values that already exist in the data with string literals
b. Transform the trans column so that it only contains either manual or auto.
Hint: Consider spark string methods and when().otherwise() chaining

In [20]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()

import pandas as pd
import numpy as np

from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [21]:
mpg.createOrReplaceTempView("mpg")

### a. Create 1 column of output that contains a message like the one below for each record:
The 1999 audi a4 has a 4 cylinder engine.

In [23]:
spark.sql(
    """
SELECT hwy, cty, (hwy + cty) / 2 AS avg
FROM mpg
"""
).show(5)

+---+---+----+
|hwy|cty| avg|
+---+---+----+
| 29| 18|23.5|
| 29| 21|25.0|
| 31| 20|25.5|
| 30| 21|25.5|
| 26| 16|21.0|
+---+---+----+
only showing top 5 rows



In [63]:
# Note: The pyspark avg and mean functions are aliases of eachother
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_extract, regexp_replace
from pyspark.sql.functions import when

In [74]:
mpg.select(concat(lit('The '), 
                  mpg.year,
                  lit(' '),
                  mpg.manufacturer,
                  lit(' '),
                  mpg.model,
                  lit(' '),
                 lit('has a '),
                 mpg.cyl,
                 lit(' cylinder egine.')).alias('message')).show(5, truncate=False)

+----------------------------------------+
|message                                 |
+----------------------------------------+
|The 1999 audi a4 has a 4 cylinder egine.|
|The 1999 audi a4 has a 4 cylinder egine.|
|The 2008 audi a4 has a 4 cylinder egine.|
|The 2008 audi a4 has a 4 cylinder egine.|
|The 1999 audi a4 has a 6 cylinder egine.|
+----------------------------------------+
only showing top 5 rows



## b. Transform the trans column so that it only contains either manual or auto.
Hint: Consider spark string methods and when().otherwise() chaining

In [55]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [75]:
mpg.select(mpg.trans,when(mpg.trans.like('auto%'), 'auto')
          .otherwise('manual').alias('manual_auto')).show(5)

+----------+-----------+
|     trans|manual_auto|
+----------+-----------+
|  auto(l5)|       auto|
|manual(m5)|     manual|
|manual(m6)|     manual|
|  auto(av)|       auto|
|  auto(l5)|       auto|
+----------+-----------+
only showing top 5 rows



## Exercise 3:
Load the tips dataset as a spark dataframe.
### a. What percentage of observations are smokers?
Hint: .groupBy() and .withColumn() are useful functions here
### b. Create a column that contains the tip percentage
Hint: .withColumn() is useful here
### c. Calculate the average tip percentage for each combination of sex and smoker.
Hint: Chain additional functions off the answer to part b

In [78]:
from pydataset import data
tips = spark.createDataFrame(data("tips"))
tips

DataFrame[total_bill: double, tip: double, sex: string, smoker: string, day: string, time: string, size: bigint]

### a. What percentage of observations are smokers?
Hint: .groupBy() and .withColumn() are useful functions here

In [79]:
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [126]:
tips.groupBy("smoker").count().withColumn("percent", col("count") / tips.count()).show()

+------+-----+-------------------+
|smoker|count|            percent|
+------+-----+-------------------+
|    No|  151| 0.6188524590163934|
|   Yes|   93|0.38114754098360654|
+------+-----+-------------------+



In [132]:
tips.groupBy(tips.smoker).agg(count(tips.smoker).alias('counter')).withColumn('percent', col('counter') / tips.count()).show()

+------+-------+-------------------+
|smoker|counter|            percent|
+------+-------+-------------------+
|    No|    151| 0.6188524590163934|
|   Yes|     93|0.38114754098360654|
+------+-------+-------------------+



### b. Create a column that contains the tip percentage
Hint: .withColumn() is useful here

In [134]:
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [148]:
tips.withColumn('tip_percentage', round((tips.tip / tips.total_bill)*100,2)).show()

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|          5.94|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|         16.05|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|         16.66|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|         13.98|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|         14.68|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|         18.62|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|         22.81|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|         11.61|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|         13.03|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|         21.85|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|         16.65|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|         14.18|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        

### c. Calculate the average tip percentage for each combination of sex and smoker.
Hint: Chain additional functions off the answer to part b

In [150]:
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [151]:
tips.withColumn('tip_percentage', round((tips.tip / tips.total_bill)*100,2)).groupBy('sex').pivot('smoker').agg(mean('tip_percentage')).show()

+------+------------------+------------------+
|   sex|                No|               Yes|
+------+------------------+------------------+
|Female|15.691111111111113|18.214545454545455|
|  Male| 16.06659793814433|15.276666666666667|
+------+------------------+------------------+

